In [1]:
from openstack import connection
from random import randint
import copy


In [11]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn


def get_servers(conn):
    a = [i for i in conn.compute.servers(all_tenants=True)]
    return a

######

def create_test_data(fdict,gnum,maxg,ming):
    fdict=dict(fdict)
    ng = 1
    mn = 1
    returnDict_groups_vm = {}
    returnDict_name_vm   = {}
    while ng < gnum+1:
        returnDict_groups_vm[ng]={}
        vmn=randint(ming,maxg)
        for i in range(1,vmn+1):
            res = list(fdict[randint(1,len(fdict.keys()))])
            res.append(mn)
            returnDict_groups_vm[ng][i]= res
            returnDict_name_vm[mn] = {}
            returnDict_name_vm[mn]['name'] = "gr"+str(ng)+"-name-"+str(i)
            returnDict_name_vm[mn]['vcpu']=res[0]
            returnDict_name_vm[mn]['ram']=res[1]
            returnDict_name_vm[mn]['groupid']=ng
            mn=mn+1
        ng = ng + 1
    
    return returnDict_groups_vm, returnDict_name_vm, mn-1
 
#####  
    
def create_flavor_dict(maxram=1000000):
    fl = [[conn.compute.get_flavor(flavor=fId).vcpus, conn.compute.get_flavor(flavor=fId).ram]\
               for fId in set([y.flavor['id'] for y in servers_list if y.flavor['id'] != '1']) \
               if conn.compute.get_flavor(flavor=fId).ram < maxram]
    flu = [list(x) for x in set(tuple(x) for x in fl)]
    fd ={}
    for i in range(0,len(flu)):
        fd[i+1]=flu[i]
    return fd


#####


def create_hypervisor_dict(noz=3,noh=10,hcpu=1,hram=1024):
    rd={}
    hprefix='CMP'
    min_h_in_az = noh // noz
    hnumber=1
    aznumber=1
    while aznumber < noz + 1:
        
        #print(hnumber * aznumber)
        if hnumber < min_h_in_az * aznumber:
            rd[hnumber]={'name':hprefix+str(hnumber),\
                         'az':aznumber,\
                         'free_cpu':hcpu,\
                         'free_ram':hram, \
                         'perc_ram' : 1,\
                         'perc_cpu' : 1,\
                         'vm_count' : 0}
            hnumber = hnumber + 1
            #print('1',hnumber,aznumber)
        elif hnumber > noh:
            #print('end',hnumber)
            break
        elif aznumber == noz:
            rd[hnumber]={'name':hprefix+str(hnumber),\
                         'az':aznumber,\
                         'free_cpu':hcpu,\
                         'free_ram':hram,\
                         'perc_ram' : 1,\
                         'perc_cpu' : 1,\
                         'vm_count' : 0}
            hnumber = hnumber + 1
            #print('2',hnumber,aznumber)
        else:
            aznumber = aznumber + 1
    return rd, copy.deepcopy(rd)


#######

def create_group_in_use_dict(gdict,noz=3):
    rd={}
    for i in gDict.keys():
        rd[i]={}
        rd[i]['vmtotal']=len(gDict[i].keys())
        rd[i]['az']={}
        rd[i]['hyp']={}
        for az in range(1,noz+1):
            rd[i]['az'][az]=0
        for h in range(1,number_of_hyp+1):
            rd[i]['hyp'][h]=0
    return rd, copy.deepcopy(rd)

#######

def count_over (h,v):
    h=dict(h)
    v=dict(v)
    vm_vcpu_sum = sum([v[i]['vcpu'] for i in v.keys()])
    vm_ram_sum = sum([v[i]['ram'] for i in v.keys()])
    hyper_vcpu_sum = sum([h[i]['free_cpu'] for i in h.keys()])
    hyper_ram_sum = sum([h[i]['free_ram'] for i in h.keys()])
    
    return [vm_vcpu_sum, vm_ram_sum], [hyper_vcpu_sum,hyper_ram_sum], [vm_vcpu_sum / hyper_vcpu_sum, vm_ram_sum/hyper_ram_sum] 





def calculate_after_loc(v,h,hdictinuse,gdictinuse):
    ### Изменения во временных словарях после локации
    hdictinuse[h]['free_cpu'] = (hdictinuse[h]['free_cpu'] - vDict[v]['vcpu']) 
    hdictinuse[h]['perc_cpu'] = (hdictinuse[h]['free_cpu'] - vDict[v]['vcpu']) / hyper_cpu
    hdictinuse[h]['free_ram'] = (hdictinuse[h]['free_ram'] - vDict[v]['ram']) 
    hdictinuse[h]['perc_ram'] = (hdictinuse[h]['free_ram'] - vDict[v]['ram']) / hyper_ram
    hdictinuse[h]['vm_count'] = (hdictinuse[h]['vm_count'] + 1)
    gdictinuse[vDict[v]['groupid']]['az'][hDict[h]['az']] = gdictinuse[vDict[v]['groupid']]['az'][hDict[h]['az']] + 1 
    gdictinuse[vDict[v]['groupid']]['hyp'][h] = gdictinuse[vDict[v]['groupid']]['hyp'][h] + 1

    
    
def get_hid_by_score(hd,gd,vid):
    rd={}
    for h in hDict.keys():
        group = vDict[vid]['groupid']
        az = hDict[h]['az']
        vmt = gd[group]['vmtotal']
        machine_in_az = gd[group]['az'][az]
        machine_in_hyp = gd[group]['hyp'][h]
        #print((hd[h]['free_ram'] - vDict[vid]['ram']) / hyper_ram)
        if (hd[h]['free_ram'] - vDict[vid]['ram']) / hyper_ram < 0.05:
            score=-10
            score_az = (1 - machine_in_az / vmt)
            score_hyp = (1 - machine_in_hyp / vmt)
        else:  
            score_sum = (0.15 * hd[h]['perc_cpu'] + hd[h]['perc_ram'])/2
            score_az = (1 - machine_in_az / vmt)
            score_hyp = (1 - machine_in_hyp / vmt)
            score = 1.5 * score_sum * 1.1 * score_az * 1.2 * score_hyp
        rd[h]={#'mply': score_mply,\
                           'score_az':score_az,\
                           'score_hyp':score_hyp, 
                           'score': score,\
                           'id' : h,\
                           'ram': hd[h]['free_ram'],\
                           'cpu': hd[h]['free_cpu'],\
                           'vm_count' : hd[h]['vm_count']
                          }
    ll = [[rd[i]['id'], rd[i]['score']] for i in rd.keys()]
    winner = sorted(ll, key = lambda x: float(x[1]), reverse=True)[0]
    return winner[0], winner[1]




In [3]:
## ask OpenStack about servers and flavors
conn = connection_to_cluster('admin')
servers_list = get_servers(conn)
flavor_dict = create_flavor_dict(65357)

In [12]:
## fake variables
number_of_groups = 200
max_vm_in_group = 15
min_vm_in_group = 2
number_of_hyp = 76
hyper_cpu = 56
hyper_ram = 512000


In [13]:
## create test data - dcit of groups and dict of vms
gDict, vDict, vm_count = create_test_data(flavor_dict,number_of_groups,max_vm_in_group,min_vm_in_group)
hDict, hDict_in_use = create_hypervisor_dict(noz=3,noh=number_of_hyp,hram=hyper_ram,hcpu=hyper_cpu)
gDictLocation, gDictLocation_in_use = create_group_in_use_dict(gDict,noz=3)


vm_res_total_list, hyper_res_total_list, over_total_list = count_over(hDict,vDict)

In [14]:
#vm_count

In [15]:
print("Всего машин:", vm_count)
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", hyper_cpu, "RAM :", hyper_ram, "MB")
print("Всего ВУ:", number_of_hyp)
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

Всего машин: 1706
Необходимо машинам vCPU : 10494 , RAM : 30597664 MB
На одном ВУ CPU : 56 RAM : 512000 MB
Всего ВУ: 76
Суммарно на ВУ : 4256 , RAM : 38912000 MB
Суммарная подписка по CPU : 2.4656954887218046 , RAM : 0.7863297697368421


In [16]:
scoring_list=[]
iteration=0
for r,i in sorted([[vDict[i]['ram'],i] for i in vDict.keys()], key = lambda x: int(x[0]),reverse=True):
    iteration=iteration + 1 
    hid, hid_score = get_hid_by_score(gd=gDictLocation_in_use,hd=hDict_in_use,vid=i)
    calculate_after_loc(gdictinuse=gDictLocation_in_use,hdictinuse=hDict_in_use,h=hid,v=i)
    scoring_list.append([iteration, i, r, hid, hid_score])
    #print(i, r, hid, hid_score)

In [17]:
s = [h[4] for h in scoring_list]
print("Значения score: ")
print("Максимальный скоринг на итерации :", max(s))
print("Минимальный скоринг на итерации :", min(s))
print("Средний :", sum(s) / len(s) )


Значения score: 
Максимальный скоринг на итерации : 1.1384999999999998
Минимальный скоринг на итерации : -0.012612555803571403
Средний : 0.3230198866878473


In [18]:
l = [hDict_in_use[h]['perc_ram'] for h in hDict.keys()]
m = [hDict_in_use[h]['perc_cpu'] for h in hDict.keys()]

print("Забитость ВУ по RAM после распределения ")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ после распределения ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))

Забитость ВУ по RAM после распределения 
Самый забитый : 0.848609375
Наименее забитый : 0.748140625
Средняя загруженность : 0.7943297697368421

Перерасход vCPU относительно CPU ВУ после распределения 
Самый забитый : 2.928571428571429
Наименее забитый : 2.142857142857143
Средняя загруженность : 2.5765977443609023


In [ ]:
#vm_res_total_list, hyper_res_total_list, over_total_list

In [ ]:
#max([i for i in vDict.keys()]), vDict[max([i for i in vDict.keys()])]

In [ ]:
#vDict[1]

In [ ]:
#flavor_list_uniqs, flavor_list_uniqs[21]

In [ ]:

#hDict_in_use=copy.deepcopy(hDict)

In [ ]:
#vDict[12]['vcpu'],vDict[12]['ram']

In [ ]:
#hDict_in_use[76]['free_cpu'] = (hDict_in_use[76]['free_cpu'] - vDict[1]['vcpu']) 
#hDict_in_use[76]['perc_cpu'] = (hDict_in_use[76]['free_cpu'] - vDict[1]['vcpu']) / hyper_cpu
#hDict_in_use[76]['free_ram'] = (hDict_in_use[76]['free_ram'] - vDict[1]['ram']) 
#hDict_in_use[76]['perc_ram'] = (hDict_in_use[76]['free_ram'] - vDict[1]['ram']) / hyper_ram

In [ ]:
#hDict[75],hDict_in_use[75]

In [ ]:

for r,i in sorted([[vDict[i]['ram'],i] for i in vDict.keys()], key = lambda x: int(x[0]),reverse=True):
    hid, hid_score = get_hid_by_score(gd=gDictLocation_in_use,hd=hDict_in_use,vid=i)
    calculate_after_loc(gdictinuse=gDictLocation_in_use,hdictinuse=hDict_in_use,h=hid,v=i)
    print(i, r, hid, hid_score)

In [ ]:
#for i in vDict.keys():
#    hid, hid_score = get_hid_by_score(gd=gDictLocation_in_use,hd=hDict_in_use,vid=i)
#    calculate_after_loc(gdictinuse=gDictLocation_in_use,hdictinuse=hDict_in_use,h=hid,v=i)
#    print(i, hid, hid_score)

In [ ]:
#hDict_in_use

In [ ]:
#[[i, gDictLocation_in_use[i]['az']] for i in gDict.keys()]

In [ ]:
min(l), max(l), sum(l)/len(l)



In [ ]:
min(m), max(m), sum(m)/len(m)

In [ ]:
'''
for i in range(1,10):
    vid = randint(1,vm_count)
    hid = randint(1,number_of_hyp)
    calculate_after_loc(vid,hid,hDict_in_use,gDictLocation_in_use)
    i=i+1
'''

In [ ]:
#calculate_after_loc(1,76,hDict_in_use,gDictLocation_in_use)

In [ ]:
#[[i, gDictLocation_in_use[i]['az']] for i in gDictLocation_in_use.keys()]
#gDictLocation_in_use[1], gDictLocation_in_use[68]['az']

In [ ]:
#rd = {}
#vid = 3
'''
for hypervisor_id in hDict.keys():
    group = vDict[vid]['groupid']
    az = hDict[hypervisor_id]['az']
    vmt = gDictLocation_in_use[group]['vmtotal']
    machine_in_az = gDictLocation_in_use[group]['az'][az]
    machine_in_hyp = gDictLocation_in_use[group]['hyp'][hypervisor_id]
   # score_mply = hDict_in_use[hypervisor_id]['perc_cpu'] * hDict_in_use[hypervisor_id]['perc_ram']
    score_sum = (hDict_in_use[hypervisor_id]['perc_cpu'] + hDict_in_use[hypervisor_id]['perc_ram'])/2
    score_az = (1 - machine_in_az / vmt)
    score_hyp = (1 - machine_in_hyp / vmt)
    score = score_sum * score_az * score_hyp
    
    
    rd[hypervisor_id]={#'mply': score_mply,\
                       'score_az':score_az,\
                       'score_hyp':score_hyp, 
                       'score': score,\
                       'id' : hypervisor_id,\
                       'ram': hDict_in_use[hypervisor_id]['free_ram'],\
                       'cpu': hDict_in_use[hypervisor_id]['free_cpu'],\
                       'vm_count' : hDict_in_use[hypervisor_id]['vm_count']
                      }
'''

In [ ]:
#[ rd[i] for i in rd.keys() if rd[i]['score'] != 1 ]

In [ ]:
#ll = [ [rd[i]['id'], rd[i]['score']] for i in rd.keys()]

#ll

In [ ]:
#sorted(ll, key = lambda x: int(x[1]), reverse=True)[0][0]

In [ ]:
def get_hid_by_score(hd,gd,vid):
    for h in hDict.keys():
        group = vDict[vid]['groupid']
        az = hDict[h]['az']
        vmt = gd[group]['vmtotal']
        machine_in_az = gd[group]['az'][az]
        machine_in_hyp = gd[group]['hyp'][h]
       # score_mply = hDict_in_use[hypervisor_id]['perc_cpu'] * hDict_in_use[hypervisor_id]['perc_ram']
        score_sum = (hd[h]['perc_cpu'] + hd[h]['perc_ram'])/2
        score_az = (1 - machine_in_az / vmt)
        score_hyp = (1 - machine_in_hyp / vmt)
        score = score_sum * score_az * score_hyp


        rd[h]={#'mply': score_mply,\
                           'score_az':score_az,\
                           'score_hyp':score_hyp, 
                           'score': score,\
                           'id' : h,\
                           'ram': hd[h]['free_ram'],\
                           'cpu': hd[h]['free_cpu'],\
                           'vm_count' : hd[h]['vm_count']
                          }
    ll = [[rd[i]['id'], rd[i]['score']] for i in rd.keys()]
    return sorted(ll, key = lambda x: int(x[1]), reverse=True)[0][0]

In [ ]:
#l = [''.join(y for y in i.name if not y.isdigit()) for i in x]

In [ ]:
#len(set(l))

In [ ]:
#new_set = set([i for i in l if not any(z in i for z in ['test', 'zbx'])])

In [ ]:
## vars
#flavor_list 


